# Import Packages

In [ ]:
import nidaqmx
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib.animation

# Define Functions

In [ ]:
def CV_potential_sweep(samp_rate, samp_num_tot, buffer_size, Rm, pot_profile):
   
    ''' 
    This function writes a potential profile used for cyclic voltammetry into the ao0 output of the 
    potentiostat to set Ein, which then sets Ecell when the counter electrode is connected. 
    
    At the same time potential acquisitions are made on input channels ai0 and ai1 to measure 
    Ecell and iwRm where Rm is the current measurement resistor
    
    Inputs:
        samp_rate = DAQ sampling rate (samples/sec)
        samp_num_tot = total number of samples taken in voltage range. Determined by potential profile and sampling rate.
        buffer_size = buffer size. 
        Rm = resistance of resistor for current measurement in ohms
        pot_profile = potential profile used for the CV scan
        
    Returns:     
        total_data_WE = potential of the WE vs RE during potential sweep (ai0) 
        total_data_RM = potential drop across Rm resistor during potential sweep (ai1)
        np.array(total_data_RM)/Rm = array containing the current passing Rm resistor during the potential sweep 
        np.abs(np.arange(0, len(total_data_WE), 1)/samp_rate) = time array during the potential sweep
    
    '''  
    
        
    '''Get device name '''    
    # get a list of all devices connected
    all_devices = list(nidaqmx.system.System.local().devices)
    # get name of first device
    dev_name = all_devices[0].name
    #print(dev_name)
    
    
    
    ''' add DAQ channels and define measurement parameters'''
    with nidaqmx.Task() as task_i, nidaqmx.Task() as task_o:
        
        # add ai0 & ai1 input channels for reading potentials. add ao0 output channels for setting potential
        task_i.ai_channels.add_ai_voltage_chan(dev_name + "/ai0:1")
        task_o.ao_channels.add_ao_voltage_chan(dev_name + "/ao0", min_val=-10.0, max_val=10.0)

        # define sampling rate and total samples acquired per channel for input & output channels
        task_i.timing.cfg_samp_clk_timing(rate = samp_rate, samps_per_chan=samp_num_tot)
        task_o.timing.cfg_samp_clk_timing(rate = samp_rate, samps_per_chan=samp_num_tot) 

        # set up a digital trigger for the output channel to set the potential
        #task_o.triggers.start_trigger.cfg_dig_edge_start_trig('/'+ dev_name +'/ai/StartTrigger')

        # create empty lists to populate
        total_data_WE = [] # WE potential
        total_data_RM = [] # Rm potential

        # define output channel task. Task will only execute when the output channel trigger is activated
        task_o.write(pot_profile)# , auto_start = False)
        task_o.start() 
        
        
        
        ''' Set up plot during data acquisition'''
        # set up a 2 x 2 grid for the plot
        grid = plt.GridSpec(2, 1, wspace=0.3, hspace=0.3)
        fig = plt.figure(figsize = (14, 12))        
        
        # set right edge of plot to be at 80% of fig width and bottom to be at 20% of fig height to fit everything.
        plt.subplots_adjust(right=0.7)
        plt.subplots_adjust(bottom=0.3)
        
        #Define positions of 3 subplots
        ax1 = fig.add_subplot(grid[1, 0])
        ax2 = fig.add_subplot(grid[0, 0])
        
        # plot
        plt.ion()
        fig.show()
        fig.canvas.draw()


        '''Buffer callback function'''
        def cont_read(task_handle, every_n_samples_event_type,
                     number_of_samples, callback_data):
            
            
            '''
            Define a 'callback' function to execute when the buffer is full
            
            When this funtion is called, n=buffer_size number of samples are acquired in ai0 and ai1 and then appended 
            to the lists 'total_data_WE'and 'total_data_RM'. Then the CV plot is updated with this new data. 
             '''
            
            # Acquire 200 potential samples and store data in a temporary list
            temp_samples = task_i.read(number_of_samples_per_channel=buffer_size)
            # add acquired data to list storing all data
            total_data_WE.extend(temp_samples[0])
            total_data_RM.extend(temp_samples[1])
            
            # calculate time profile (for plotting)
            total_time_profile = np.abs(np.arange(0, len(total_data_WE), 1)/samp_rate)
            # calculate current at Rm (for plotting)
            Rm_current = np.array(total_data_RM)/Rm
          
            
            # Return size of 'total_data' and update subplots every time buffer is full
            #print(len(total_data_RM))
                        
            ax1.clear()
            ax1.set_title('Cyclic Voltammogram', fontsize = 16)
            ax1.tick_params(axis='both',which='both',direction='in',right=True, top=True)
            ax1.set_xlabel('$E_{\mathrm{cell}}$ / V', fontsize = 16)
            ax1.set_ylabel('$i_{\mathrm{w}}$ / A', fontsize = 16)
            ax1.ticklabel_format(axis = 'y', style='sci', scilimits = (-2, 3)) 
            ax1.plot(total_data_WE, Rm_current)
            
            ax2.clear()
            ax2.set_title('$E_{\mathrm{cell}}$ and $i_{\mathrm{w}}R_{\mathrm{m}}$ vs Time', fontsize = 16)
            ax2.tick_params(axis='both',which='both',direction='in',right=True, top=True)
            # ax2.tick_params(labelbottom=False) 
            ax2.set_xlabel('Time / s', fontsize = 16)
            ax2.set_ylabel('Potential / V', fontsize = 16)
            ax2.plot(total_time_profile, total_data_WE, label = '$E_{\mathrm{cell}}$')
            ax2.plot(total_time_profile, total_data_RM, label = '$i_{\mathrm{w}}R_{\mathrm{m}}$')
            ax2.legend()
            

            # redrew plot with new data
            fig.canvas.draw()
            
            # the callback function must return an integer. Can be any integer
            return 5

        
        '''
        Define buffer and specify the 'callback' function executed every time buffer is full. 
        '''
        task_i.register_every_n_samples_acquired_into_buffer_event(buffer_size, cont_read)
        

  
        # start task to read potential at inputs. This will trigger output to begin potenial sweep
        task_i.start()
            
        
        # need an input here for some reason.
        input('press Enter to end')
        
        
    # return data
    return total_data_WE, total_data_RM, np.array(total_data_RM)/Rm, np.abs(np.arange(0, len(total_data_WE), 1)/samp_rate)

# Define potential profile for CV

Code block 1 constructs the potential profile used for the CV. This potential profile consists 
of 3 sections: 

 1) a forward potential sweep from f_start_pot to f_end_pot. <br>
 2) a backwards sweep from r_start_pot to r_end_pot.l <br>
 3) a return potential sweep from return_start_pot to return_end_pot which returns the WE potential to some     point (usually the initial potential) 

A fixed buffer size is set and this must be a factor of the potential profile sample size. A hold time (h_time) is included to make sure the sample size of the potential profile is a multiple of the buffer. The total time for each section is determined by the number of samples sent to the output channel and the sampling rate. You should check the sweep rate, Rm and the bounds of the profile carefully to prevent any unwanted reactions in your CV.

In [ ]:
'''Code block 1'''
import matplotlib
%matplotlib inline

'''Basic acquisition parameters'''

# sampling rate (samples/s)  Use a multiple of 120 e.g. 3600 samples/s
samp_rate = 3600   

# scan rate (V/s)
scan_rate = 1

# Rm resistance in Ohms
Rm = 1000


'''Set up range of potential profile for CV'''
h_time = 0.2  # hold time before sweep in seconds

f_start_pot = .113  # Initial potential (V)
f_end_pot = .272    # First vertex potential (V)

r_start_pot = f_end_pot  # reverse scan should start at forward vertex potential (V)
r_end_pot = -.2   # reverse vertex potential (V)

return_start_pot = r_end_pot 
return_end_pot = f_start_pot # return to start potential (V)


'''Total time required for each section'''
f_time = np.abs((f_end_pot - f_start_pot)/scan_rate)
r_time = np.abs((r_end_pot - r_start_pot)/scan_rate)
return_time = np.abs((return_end_pot - return_start_pot)/scan_rate)

'''Potential array to be set for each section'''
h_profile = np.linspace(f_start_pot, f_start_pot, int(samp_rate*h_time) )
f_profile = np.linspace(f_start_pot, f_end_pot, int(samp_rate*f_time) )
r_profile = np.linspace(r_start_pot, r_end_pot, int(samp_rate*r_time) )
return_profile = np.linspace(return_start_pot, return_end_pot, int(samp_rate*return_time))


'''Define a buffer size and ensure the total sample number in potential profile is a multiple of the buffer_size'''
buffer_size = 1200 # must be an integer

# additional samples in the hold step to round off potential profile
additional_hold_sample = 0

# total sample size of the potential profile
samp_num_tot = additional_hold_sample + len(f_profile)+len(r_profile)+len(return_profile)+samp_rate*h_time

#calculate number of additional hold samples that are required to make samp_num_tot a multiple of the buffer size
while samp_num_tot%buffer_size != 0:
    additional_hold_sample += 1
    samp_num_tot  = additional_hold_sample + len(f_profile)+len(r_profile)+len(return_profile)+samp_rate*h_time

# recalculate hold profile
h_profile =  np.linspace(f_start_pot, f_start_pot, int(samp_rate*h_time+additional_hold_sample))


'''construct potential profile by combining each individual section'''
pot_profile = np.concatenate((h_profile, f_profile, r_profile, return_profile))
samp_num_tot = int(len(pot_profile)) # must be an integer

'''Check potential profile to be set'''
plt.title('CV Program Potential', fontsize = 16)
plt.xlabel('Time / s', fontsize = 16)
plt.ylabel('$E_{\mathrm{in}}$ / V', fontsize = 16)
plt.tick_params(axis='both',which='both',direction='in',right=True, top=True)
plt.plot(np.arange(0, len(pot_profile), 1)/samp_rate, pot_profile)

# Set the starting potential before CV

In [ ]:
'''Get device name'''
# get a list of all devices connected
all_devices = list(nidaqmx.system.System.local().devices)
# get name of first device
dev_name = all_devices[0].name
#print(dev_name)
    
''' add DAQ channels and set the WE potential to f_start_pot'''
with nidaqmx.Task() as task_i, nidaqmx.Task() as task_o:
        
# add ai0 & ai1 input channels for reading potentials. add ao0 output channels for setting potential
    task_i.ai_channels.add_ai_voltage_chan(dev_name + "/ai0:1")
    task_o.ao_channels.add_ao_voltage_chan(dev_name + "/ao0", min_val=-10.0, max_val=10.0)
    task_o.write(f_start_pot)
    task_o.start() 

# CV Measurement

Code block 2 writes the potential profile defined in code block 1 and collects the CV data. The CV plot should update in real time (every time the sample buffer is full). 

You may terminate the measurement with the enter key. Once the measurement concludes you will be asked to enter a filename for the measurement. This will be the filename used for the saved plot and data.

In [ ]:
'''Code block 2'''

import matplotlib
%matplotlib notebook
import IPython
from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 70em; width:120em}</style>"))

# Set potential profile and acquire data from CV
A = CV_potential_sweep(samp_rate, samp_num_tot, buffer_size, Rm, pot_profile)

# ask for the name of measurement this will be used for the filenames for the exported data and plots
filename = input('Name of measurement:')

# save CV plot and data once measurements is terminated
plt.savefig(filename + '.png', dpi = 300, bbox_inches='tight')

#export data
exported_data = pd.DataFrame({'WE potential': A[0], 
                    'Rm potential': A[1], 'Rm current': A[2], 'Time (s)': A[3]})
exported_data.to_csv(filename + '.txt', index = False)

            

# Importing data and replotting

Cell block 3 shows an example of importing 2 data sets and doing some correction on the data. The corrected data are then plotted together. You may choose to import multiple datasets, do some processing like normalisation and then plot everything together to compare them.  

In [ ]:
'''Cell block 3'''

# Importing data into dataframes. Import as many files as necessary.
data1 = pd.read_csv('CV1.txt')
data2 = pd.read_csv('CV2.txt')


'''#Examples of altering imported data'''
# replace the 'WE potential' column with the doubled value in data1 dataframe
data1['WE potential'] = data1['WE potential']*2 

# uncompesnated resistance in ohms
Ru = 100
# subtract the overpotential attributed to the uncompesnated resistance in data2
data2['WE potential'] = data2['WE potential'] - data2['Rm current']*Ru 


'''plot data'''
fig, ax = plt.subplots(figsize = (12, 10))
plt.title('CV data')
plt.subplots_adjust(right=0.7)
plt.subplots_adjust(bottom=0.3)

# plot data from dataframes of interest
data1.plot(kind = 'line', x = 'WE potential', y = 'Rm current', label = 'data1 modified', ax =ax)
data2.plot(kind = 'line', x = 'WE potential', y = 'Rm current', label = 'data2 modified', ax =ax)
#data3.plot(kind = 'line', x = 'WE potential', y = 'Rm current', label = file, ax =ax)

ax.set_title('Cyclic Voltammogram', fontsize = 16)
ax.set_ylabel('Current/A')
ax.tick_params(axis='both',which='both',direction='in',right=True, top=True)
ax.set_xlabel('$E_{\mathrm{cell}}$ / V', fontsize = 16)
ax.set_ylabel('$i_{\mathrm{w}}$ / A', fontsize = 16)
ax.ticklabel_format(axis = 'y', style='sci', scilimits = (-2, 3)) 
#ax.set_aspect('equal', 'box')
#ax.set_xlim((0, 500))
#ax.set_ylim((-0.03, 0.03))
plt.tight_layout()
plt.subplots_adjust(left=0.1, right=0.7, bottom=0.3, top=0.9)
plt.legend()



# Importing and plotting multiple datasets

Code block 4 imports specified data from different measurements into a dictionary and plots everything in a single graph. Enter the text files into the list named 'file_list'. Terminate the interactive window under code block 2. You can modify the dataframes in the dictionary in the same way as shown in cell block 3. 

In [ ]:
# add file names of files you want to read
file_list = ['s.txt', 'abort.txt']

# create dictionary to populate
file_dict = {}

# add relevant files to dictionary
for file in file_list:
    file_dict[file.replace('.txt', '')] = pd.read_csv(file)


'''plot all data in dictionary'''
fig, ax = plt.subplots()
plt.title('CV data')

for file in file_dict.keys():
    file_dict[file].plot(kind = 'line', x = 'WE potential', y = 'Rm current', label = file, ax =ax)
    

ax.set_title('Cyclic Voltammogram', fontsize = 16)
ax.set_ylabel('Current/A')
ax.tick_params(axis='both',which='both',direction='in',right=True, top=True)
ax.set_xlabel('$E_{\mathrm{cell}}$ / V', fontsize = 16)
ax.set_ylabel('$i_{\mathrm{w}}$ / A', fontsize = 16)
ax.ticklabel_format(axis = 'y', style='sci', scilimits = (-2, 3)) 
#ax.set_aspect('equal', 'box')
#ax.set_xlim((0, 500))
#ax.set_ylim((-0.03, 0.03))
plt.tight_layout()
plt.legend()

